In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import utils.delphesRootToPixelAvTrackList as utils
import pandas as pd
h=30

In [2]:
def plot(num, R, q, theta, xmin, xmax, ymin, ymax):
    xOuter = h*np.cos(theta)
    yOuter = h*np.sin(theta)

    # The maximum and minimum possible entry points with respect to the whole pixel sensor:
    yentrymin=-16/2+0.08125
    yentrymax=16/2-0.08125

    # Using the max and min allowed entry points, get the max and min possible beta values
    betamin = utils.getBeta(yentrymax, R, q)
    betamax = utils.getBeta(yentrymin, R, q)

    # Pick a beta within the allowed range
    beta=betamin+(betamax-betamin)*num

    # Get the y entry point
    y0, xc, yc = utils.getYentry(R, q, beta)

    x,y=utils.getCrossingPoint(xc,yc,R,q)

    gamma=utils.getGamma(xOuter, yOuter, xc, yc, R, q)

    xc_particle=xc*np.cos(gamma)-yc*np.sin(gamma)
    yc_particle=xc*np.sin(gamma)+yc*np.cos(gamma)

    fig, ax = plt.subplots(1, figsize=(5,4), dpi=200)
    circle=plt.Circle((xc, yc), R, fill=0, color='b', label = "Assumed particle track")
    circle0=plt.Circle((xc_particle, yc_particle), R, fill=0, color='g', label = "Actuall particle track")
    circle1=plt.Circle((0, 0), h, fill=0, color='k', label="Barrel")
    ax.add_patch(circle)
    ax.add_patch(circle0)
    ax.add_patch(circle1)
    ax.scatter(h,-y0, label="Point where particle crosses sensor (based on assumed sensor location)")
    ax.scatter(xc,yc,c='k')
    ax.scatter(x,y,c='m', label="Point where particle crosses r = 30 mm (based on assumed sensor location)")
    ax.scatter(xOuter,yOuter,c='orange', label="Actual location where particle crossed r = 30 mm")
    ax.quiver(h,-y0, np.cos(beta-np.pi/2), np.sin(beta-np.pi/2), angles='xy')
    ax.plot((h,h), (-8,8),color='r', label="Assumed sensor location")
    #ax.axhline(y=h, xmin=-8, xmax=8, color='black')
    ax.set_xlim(xmin,xmax)
    ax.set_ylim(ymin,ymax)
    ax.set_title(f"Gamma={round(gamma/np.pi,2)}\u03C0, y-entry={round(y0,3)}")
    plt.gca().set_aspect('equal')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    fig.canvas.draw()



In [3]:
widgets.interact(plot, num=(0, 1, .01), R=widgets.IntSlider(min=100, max=300, step=1, value=200), q=[-1,1], theta=(0, 2*np.pi, 0.1), xmin=widgets.IntSlider(min=-35, max=35, step=1, value=-35), xmax=widgets.IntSlider(min=0, max=35, step=1, value=35), ymin=widgets.IntSlider(min=-35, max=35, step=1, value=-35), ymax=widgets.IntSlider(min=0, max=35, step=1, value=35))

interactive(children=(FloatSlider(value=0.0, description='num', max=1.0, step=0.01), IntSlider(value=200, desc…

<function __main__.plot(num, R, q, theta, xmin, xmax, ymin, ymax)>

In [4]:
def plot(q, pT, theta, xmin, xmax, ymin, ymax):
    temp = pd.DataFrame()

    xOuter = h*np.cos(theta)
    yOuter = h*np.sin(theta)
    temp["Track.XOuter"]=np.array([xOuter])
    temp["Track.YOuter"]=np.array([yOuter])

    temp["Track.Charge"]=np.array([q])
    temp["Track.PT"] = np.array([pT])
    temp["Track.PID"] = np.array([q*211])

    temp["Track.EtaOuter"] = np.array([1])

    alpha, beta, yentry, gamma, xc, yc, xc_actual, yc_actual, R, nx, ny, nz = utils.ArrangeSensor(temp,True)

    beta=beta[0]
    yentry=yentry[0]
    gamma=gamma[0]
    xc=xc[0]
    yc=yc[0]
    xc_actual=xc_actual[0]
    yc_actual=yc_actual[0]
    R=R[0]

    fig, ax = plt.subplots(1, figsize=(5,4), dpi=200)
    circle=plt.Circle((xc, yc), R, fill=0, color='b', label = "Assumed particle track")
    circle0=plt.Circle((xc_actual, yc_actual), R, fill=0, color='g', label = "Actuall particle track")
    circle1=plt.Circle((0, 0), h, fill=0, color='k', label="Barrel")
    ax.add_patch(circle)
    ax.add_patch(circle0)
    ax.add_patch(circle1)
    ax.scatter(h,-yentry, label="Point where particle crosses sensor  (based on assumed sensor location)")
    #ax.scatter(xc,yc,c='b')
    #ax.scatter(x,y,c='m', label="Point where particle crosses r = 30 mm (based on assumed sensor location)")
    ax.scatter(xOuter,yOuter,c='orange', label="Actual location where particle crossed r = 30 mm")
    ax.quiver(h,-yentry, np.sin(beta), np.cos(beta), angles='xy')
    ax.quiver(h,-yentry, np.sin(beta), np.cos(beta), angles='xy')
    ax.plot((h,h), (-8,8),color='r', label="Assumed sensor location")
    #ax.axhline(y=h, xmin=-8, xmax=8, color='black')
    ax.set_xlim(xmin,xmax)
    ax.set_ylim(ymin,ymax)
    ax.set_title(f"Gamma={round(gamma/np.pi,2)}\u03C0, y-entry={round(yentry,3)}")
    plt.gca().set_aspect('equal')
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    fig.canvas.draw()



In [5]:

widgets.interact(plot, q=[-1,1], pT= (1,2,.1),theta=(0, 2*np.pi, 0.1), xmin=widgets.IntSlider(min=-35, max=35, step=1, value=-35), xmax=widgets.IntSlider(min=0, max=35, step=1, value=35), ymin=widgets.IntSlider(min=-35, max=35, step=1, value=-35), ymax=widgets.IntSlider(min=0, max=35, step=1, value=35))

interactive(children=(Dropdown(description='q', options=(-1, 1), value=-1), FloatSlider(value=1.0, description…

<function __main__.plot(q, pT, theta, xmin, xmax, ymin, ymax)>